#Data Preprocessing


In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score
from scipy.stats import chi2_contingency
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, precision_recall_fscore_support
from sklearn.linear_model import LogisticRegression
import warnings
import os
import joblib

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
a1 = pd.read_csv("/content/drive/MyDrive/Credit_risk/case_study1.csv")
a2 = pd.read_csv("/content/drive/MyDrive/Credit_risk/case_study2.csv")

In [ ]:
df1 = a1.copy()
df2 = a2.copy()

In [9]:
df1 = df1.loc[df1['Age_Oldest_TL'] != -99999]
columns_to_be_removed = []
for i in df2.columns:
  if df2.loc[df2[i] == -99999].shape[0] > 10000:
    columns_to_be_removed.append(i)

In [10]:
df2 = df2.drop(columns_to_be_removed, axis= 1)
for i in df2.columns:
  df2 = df2.loc[df2[i] != -99999]

In [11]:
for i in list(df1.columns):
  if i in list(df2.columns):
    print(i)

PROSPECTID


In [12]:
df = pd.merge(df1, df2, how = 'inner', left_on = ['PROSPECTID'], right_on = ['PROSPECTID'])

In [13]:
for i in df.columns:
    if df[i].dtype == 'object':
        print(i)

MARITALSTATUS
EDUCATION
GENDER
last_prod_enq2
first_prod_enq2
Approved_Flag


In [14]:
for i in ['MARITALSTATUS', 'EDUCATION','GENDER','last_prod_enq2', 'first_prod_enq2']:
  chi2, pval, _, _ = chi2_contingency(pd.crosstab(df[i], df['Approved_Flag']))
  print(i,'---', pval)

MARITALSTATUS --- 3.578180861038862e-233
EDUCATION --- 2.6942265249737532e-30
GENDER --- 1.907936100186563e-05
last_prod_enq2 --- 0.0
first_prod_enq2 --- 7.84997610555419e-287


In [15]:
numeric_columns = []
for i in df.columns:
  if df[i].dtype != 'object' and i not in ['PROSPECTID','Approved_Flag']:
    numeric_columns.append(i)

In [16]:
vif_data = df[numeric_columns]
total_columns = vif_data.shape[1]
columns_to_be_kept = []
column_index = 0

In [17]:
for i in range(0, total_columns):
  vif_value = variance_inflation_factor(vif_data, column_index)
  print(column_index, '----', vif_value)
  if vif_value <= 6:
    columns_to_be_kept.append(numeric_columns[i])
    column_index = column_index+1
  else:
    vif_data = vif_data.drop([numeric_columns[i]], axis = 1)

/usr/local/lib/python3.10/dist-packages/statsmodels/stats/outliers_influence.py:197: RuntimeWarning: divide by zero encountered in scalar divide
  vif = 1. / (1. - r_squared_i)


0 ---- inf


/usr/local/lib/python3.10/dist-packages/statsmodels/stats/outliers_influence.py:197: RuntimeWarning: divide by zero encountered in scalar divide
  vif = 1. / (1. - r_squared_i)


0 ---- inf
0 ---- 11.320180023967996
0 ---- 8.363698035000336
0 ---- 6.520647877790928
0 ---- 5.149501618212625
1 ---- 2.611111040579735


/usr/local/lib/python3.10/dist-packages/statsmodels/stats/outliers_influence.py:197: RuntimeWarning: divide by zero encountered in scalar divide
  vif = 1. / (1. - r_squared_i)


2 ---- inf
2 ---- 1788.7926256209232
2 ---- 8.601028256477228
2 ---- 3.832800792153077
3 ---- 6.099653381646723
3 ---- 5.581352009642766
4 ---- 1.985584353098778


/usr/local/lib/python3.10/dist-packages/statsmodels/stats/outliers_influence.py:197: RuntimeWarning: divide by zero encountered in scalar divide
  vif = 1. / (1. - r_squared_i)


5 ---- inf
5 ---- 4.80953830281934
6 ---- 23.270628983464636
6 ---- 30.595522588100053
6 ---- 4.384346405965583
7 ---- 3.0646584155234238
8 ---- 2.898639771299251
9 ---- 4.377876915347324
10 ---- 2.207853583695844
11 ---- 4.916914200506864
12 ---- 5.214702030064725
13 ---- 3.3861625024231476
14 ---- 7.840583309478997
14 ---- 5.255034641721434


/usr/local/lib/python3.10/dist-packages/statsmodels/stats/outliers_influence.py:197: RuntimeWarning: divide by zero encountered in scalar divide
  vif = 1. / (1. - r_squared_i)


15 ---- inf
15 ---- 7.380634506427238
15 ---- 1.4210050015175733
16 ---- 8.083255010190316
16 ---- 1.6241227524040114
17 ---- 7.257811920140003
17 ---- 15.59624383268298
17 ---- 1.825857047132431
18 ---- 1.5080839450032664
19 ---- 2.172088834824578
20 ---- 2.6233975535272274
21 ---- 2.2959970812106176
22 ---- 7.360578319196446
22 ---- 2.1602387773102567
23 ---- 2.8686288267891467
24 ---- 6.458218003637272
24 ---- 2.8474118865638247
25 ---- 4.753198156284083
26 ---- 16.22735475594825
26 ---- 6.424377256363877
26 ---- 8.887080381808678
26 ---- 2.3804746142952653
27 ---- 8.60951347651454
27 ---- 13.06755093547673
27 ---- 3.500040056654653
28 ---- 1.9087955874813773
29 ---- 17.006562234161628
29 ---- 10.730485153719197
29 ---- 2.3538497522950275
30 ---- 22.10485591513649
30 ---- 2.7971639638512924
31 ---- 3.424171203217696
32 ---- 10.175021454450922
32 ---- 6.408710354561292
32 ---- 1.001151196262563
33 ---- 3.069197305397273
34 ---- 2.8091261600643724
35 ---- 20.249538381980678
35 ---- 15

In [19]:
from scipy.stats import f_oneway

columns_to_be_kept_numerical = []

for i in columns_to_be_kept:
  a = list(df[i])
  b = list(df['Approved_Flag'])

  group_P1 = [value for value, group in zip(a, b) if group == 'P1']
  group_P2 = [value for value, group in zip(a, b) if group == 'P2']
  group_P3 = [value for value, group in zip(a, b) if group == 'P3']
  group_P4 = [value for value, group in zip(a, b) if group == 'P4']

  f_statistic, p_value = f_oneway(group_P1, group_P2, group_P3, group_P4)

  if p_value <= 0.05:
    columns_to_be_kept_numerical.append(i)

In [20]:
features = columns_to_be_kept_numerical + ['MARITALSTATUS', 'EDUCATION', 'GENDER', 'last_prod_enq2', 'first_prod_enq2' ]
df = df[features + ['Approved_Flag']]

In [22]:
df['EDUCATION'].unique()

array(['12TH', 'GRADUATE', 'SSC', 'POST-GRADUATE', 'UNDER GRADUATE',
       'OTHERS', 'PROFESSIONAL'], dtype=object)

In [21]:
df['MARITALSTATUS'].unique()

array(['Married', 'Single'], dtype=object)

In [23]:
df['GENDER'].unique()

array(['M', 'F'], dtype=object)

In [24]:
df['last_prod_enq2'].unique()

array(['PL', 'ConsumerLoan', 'AL', 'CC', 'others', 'HL'], dtype=object)

In [25]:
df['first_prod_enq2'].unique()

array(['PL', 'ConsumerLoan', 'others', 'AL', 'HL', 'CC'], dtype=object)

In [26]:
df.loc[df['EDUCATION'] == 'SSC',['EDUCATION']]              = 1
df.loc[df['EDUCATION'] == '12TH',['EDUCATION']]             = 2
df.loc[df['EDUCATION'] == 'GRADUATE',['EDUCATION']]         = 3
df.loc[df['EDUCATION'] == 'UNDER GRADUATE',['EDUCATION']]   = 3
df.loc[df['EDUCATION'] == 'POST-GRADUATE',['EDUCATION']]    = 4
df.loc[df['EDUCATION'] == 'OTHERS',['EDUCATION']]           = 1
df.loc[df['EDUCATION'] == 'PROFESSIONAL',['EDUCATION']]     = 3
df['EDUCATION'].value_counts()
df['EDUCATION'] = df['EDUCATION'].astype(int)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42064 entries, 0 to 42063
Data columns (total 43 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   pct_tl_open_L6M            42064 non-null  float64
 1   pct_tl_closed_L6M          42064 non-null  float64
 2   Tot_TL_closed_L12M         42064 non-null  int64  
 3   pct_tl_closed_L12M         42064 non-null  float64
 4   Tot_Missed_Pmnt            42064 non-null  int64  
 5   CC_TL                      42064 non-null  int64  
 6   Home_TL                    42064 non-null  int64  
 7   PL_TL                      42064 non-null  int64  
 8   Secured_TL                 42064 non-null  int64  
 9   Unsecured_TL               42064 non-null  int64  
 10  Other_TL                   42064 non-null  int64  
 11  Age_Oldest_TL              42064 non-null  int64  
 12  Age_Newest_TL              42064 non-null  int64  
 13  time_since_recent_payment  42064 non-null  int

In [27]:
df_encoded = pd.get_dummies(df, columns=['MARITALSTATUS','GENDER', 'last_prod_enq2' ,'first_prod_enq2'])
df_encoded.info()
k = df_encoded.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42064 entries, 0 to 42063
Data columns (total 55 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   pct_tl_open_L6M               42064 non-null  float64
 1   pct_tl_closed_L6M             42064 non-null  float64
 2   Tot_TL_closed_L12M            42064 non-null  int64  
 3   pct_tl_closed_L12M            42064 non-null  float64
 4   Tot_Missed_Pmnt               42064 non-null  int64  
 5   CC_TL                         42064 non-null  int64  
 6   Home_TL                       42064 non-null  int64  
 7   PL_TL                         42064 non-null  int64  
 8   Secured_TL                    42064 non-null  int64  
 9   Unsecured_TL                  42064 non-null  int64  
 10  Other_TL                      42064 non-null  int64  
 11  Age_Oldest_TL                 42064 non-null  int64  
 12  Age_Newest_TL                 42064 non-null  int64  
 13  t

#Model Training


In [28]:
# XGBoost
import xgboost as xgb
from sklearn.preprocessing import LabelEncoder

xgb_classifier = xgb.XGBClassifier(objective='multi:softmax',  num_class=4)
y = df_encoded['Approved_Flag']
x = df_encoded. drop ( ['Approved_Flag'], axis = 1 )


label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)


x_train, x_test, y_train, y_test = train_test_split(x, y_encoded, test_size=0.2, random_state=42)

xgb_classifier.fit(x_train, y_train)
y_pred = xgb_classifier.predict(x_test)

accuracy = accuracy_score(y_test, y_pred)
print ()
print(f'Accuracy: {accuracy:.2f}')
print ()

precision, recall, f1_score, _ = precision_recall_fscore_support(y_test, y_pred)

for i, v in enumerate(['p1', 'p2', 'p3', 'p4']):
    print(f"Class {v}:")
    print(f"Precision: {precision[i]}")
    print(f"Recall: {recall[i]}")
    print(f"F1 Score: {f1_score[i]}")
    print()


Accuracy: 0.78

Class p1:
Precision: 0.823906083244397
Recall: 0.7613412228796844
F1 Score: 0.7913890312660173

Class p2:
Precision: 0.8255418233924413
Recall: 0.913577799801784
F1 Score: 0.8673315769665036

Class p3:
Precision: 0.4756380510440835
Recall: 0.30943396226415093
F1 Score: 0.3749428440786465

Class p4:
Precision: 0.7342386032977691
Recall: 0.7356656948493683
F1 Score: 0.7349514563106796



In [29]:
# Logistic Regression
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(max_iter=200)
model.fit(x_train, y_train)
y_pred = model.predict(x_test)
accuracy = accuracy_score(y_test, y_pred)
print ()
print(f"Accuracy: {accuracy:.2f}")
print ()

precision, recall, f1_score, _ = precision_recall_fscore_support(y_test, y_pred)

for i, v in enumerate(['p1', 'p2', 'p3', 'p4']):
    print(f"Class {v}:")
    print(f"Precision: {precision[i]}")
    print(f"Recall: {recall[i]}")
    print(f"F1 Score: {f1_score[i]}")
    print()


Accuracy: 0.62

Class p1:
Precision: 0.5552665799739922
Recall: 0.42110453648915186
F1 Score: 0.4789680314077397

Class p2:
Precision: 0.6267942583732058
Recall: 0.934786917740337
F1 Score: 0.7504176943273132

Class p3:
Precision: 0.0
Recall: 0.0
F1 Score: 0.0

Class p4:
Precision: 0.39473684210526316
Recall: 0.043731778425655975
F1 Score: 0.07874015748031497



/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [30]:
from sklearn.preprocessing  import StandardScaler
columns_to_be_scaled = ['Age_Oldest_TL', 'time_since_recent_payment', 'max_recent_level_of_deliq', 'recent_level_of_deliq', 'time_since_recent_enq', 'NETMONTHLYINCOME', 'Time_With_Curr_Empr']

for i in columns_to_be_scaled:
  column_data = df_encoded[i].values.reshape(-1,1)
  scaler = StandardScaler()
  scaled_column = scaler.fit_transform(column_data)
  df_encoded[i] = scaled_column

In [31]:
from sklearn.model_selection import GridSearchCV

model = LogisticRegression(max_iter=200)
y = df_encoded['Approved_Flag']
x = df_encoded.drop(['Approved_Flag'], axis = 1)
label_encoder = LabelEncoder()
y_encoded =  label_encoder.fit_transform(y)
x_train, x_test, y_train, y_test = train_test_split(x, y_encoded, test_size = 0.2, random_state = 42)
param_grid = {
    'C': [0.1, 1, 10, 100],  # Inverse of regularization strength
    'penalty': ['l1', 'l2'],  # Regularization type
    'solver': ['liblinear']   # Solver choice (liblinear supports l1 and l2 penalties)
}
grid_search = GridSearchCV(model, param_grid, cv=5, scoring='accuracy')
grid_search.fit(x_train, y_train)
y_pred = grid_search.predict(x_test)
print(classification_report(y_test, y_pred))


model.fit(x_train, y_train)
y_pred = model.predict(x_test)
accuracy = accuracy_score(y_test, y_pred)
print ()
print(f"Accuracy: {accuracy:.2f}")
print ()

precision, recall, f1_score, _ = precision_recall_fscore_support(y_test, y_pred)

for i, v in enumerate(['p1', 'p2', 'p3', 'p4']):
    print(f"Class {v}:")
    print(f"Precision: {precision[i]}")
    print(f"Recall: {recall[i]}")
    print(f"F1 Score: {f1_score[i]}")
    print()

              precision    recall  f1-score   support

           0       0.82      0.66      0.73      1014
           1       0.73      0.96      0.83      5045
           2       0.40      0.01      0.02      1325
           3       0.69      0.65      0.67      1029

    accuracy                           0.74      8413
   macro avg       0.66      0.57      0.56      8413
weighted avg       0.69      0.74      0.67      8413


Accuracy: 0.75

Class p1:
Precision: 0.8231850117096019
Recall: 0.6932938856015779
F1 Score: 0.7526766595289079

Class p2:
Precision: 0.7647153684889534
Recall: 0.9399405351833499
F1 Score: 0.843322070069358

Class p3:
Precision: 0.40063091482649843
Recall: 0.09584905660377359
F1 Score: 0.1546894031668697

Class p4:
Precision: 0.6791546589817483
Recall: 0.6870748299319728
F1 Score: 0.6830917874396134



/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
